In [31]:
import pandas as pd
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import numpy as np
from plotly.graph_objs import Scatter, Figure, Layout, Histogram, Box
import plotly.graph_objs as go
import plotly.figure_factory as ff
import spacy
init_notebook_mode(connected=True)

In [33]:
nlp = spacy.load('fr_core_news_sm', disable=["tagger", "parser", "ner"])

In [2]:
df_democratie = pd.read_csv("data/DEMOCRATIE_ET_CITOYENNETE.csv")
df_fiscalite = pd.read_csv("data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv")
df_transition = pd.read_csv("data/LA_TRANSITION_ECOLOGIQUE.csv")
df_services = pd.read_csv("data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
tweeter = pd.read_csv("data/tweets.csv")

In [4]:
tweeter.head()

,fullname,username,date,answerTo,message,n_answers,n_likes,n_retweet
0,u ganciu _ matricule 2031,@uganciu,1.548201e+09,\n\n\n En réponse à @KersausonDe @xavier_al...,"Ça situe assez bien le niveau de ""l'expertise""...",NaN,NaN,1
1,Lacydon,@cm_Lacydon,1.548201e+09,\n En réponse à @franceinfo\n,Sur les synthèses nécessaires à chaque étape #...,NaN,NaN,NaN
2,Arts Culture et Paix 🌐 🙏,@lafete33,1.548200e+09,\n\n\n En réponse à @GG_RMC @Cyrilhanouna @...,"OUPS... l'argument de ""Galilée"" pour étayer se...",1.0,1,NaN
3,Julien Ailloud,@JulienAilloud,1.548200e+09,NaN,Rien d'étonnant : blablater dans le vent pour ...,NaN,NaN,NaN
4,Marien Lovichi,@MarienLovichi,1.548199e+09,NaN,"Vous habitez à #Dijon ? Vendredi, le #MoDem a...",1.0,7,4


In [5]:
import time
import datetime

In [6]:
dt = datetime.datetime.strptime(df_democratie["createdAt"][0], '%Y-%m-%d %H:%M:%S')

In [7]:
unixtime = time.mktime(dt.timetuple())

In [8]:
for df in [df_democratie, df_fiscalite, df_services, df_transition]:
    times = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
             for x in df["createdAt"]]
    df["UNIXTIME"] = times

In [9]:
metadata = list(df_democratie.columns[:10]) + list(df_democratie.columns[-1:])
del metadata[1]

In [10]:
import nltk
from nltk.tokenize import word_tokenize

In [36]:
words_time_gdn = dict()
for df in [df_democratie, df_fiscalite, df_services, df_transition]:
    for i, row in df.iterrows():
        time_gdn = row["UNIXTIME"]
        for col in df.columns:
            if col in metadata:
                continue
            if not isinstance(row[col], str):
                continue
            # for word in word_tokenize(row[col], language='french'):
            for word in nlp(row[col]):
                # word = word.lower()
                word = word.lemma_
                if word in words_time_gdn:
                    words_time_gdn[word].append(time_gdn)
                else:
                    words_time_gdn[word] = [time_gdn]

In [12]:
iplot([go.Histogram(x=words_time_gdn["80"])])

In [13]:
datetime.datetime.fromtimestamp(1550505603)

datetime.datetime(2019, 2, 18, 17, 0, 3)

In [14]:
def to_datetime(x):
    if np.isnan(x):
        return x
    else:
        return datetime.datetime.fromtimestamp(int(x))

In [15]:
times_tweeter = [to_datetime(x) for x in tweeter["date"]]
tweeter["UNIXTIME"] = times_tweeter

In [16]:
tweeter.head()

,fullname,username,date,answerTo,message,n_answers,n_likes,n_retweet,UNIXTIME
0,u ganciu _ matricule 2031,@uganciu,1.548201e+09,\n\n\n En réponse à @KersausonDe @xavier_al...,"Ça situe assez bien le niveau de ""l'expertise""...",NaN,NaN,1,2019-01-23 00:55:35
1,Lacydon,@cm_Lacydon,1.548201e+09,\n En réponse à @franceinfo\n,Sur les synthèses nécessaires à chaque étape #...,NaN,NaN,NaN,2019-01-23 00:54:26
2,Arts Culture et Paix 🌐 🙏,@lafete33,1.548200e+09,\n\n\n En réponse à @GG_RMC @Cyrilhanouna @...,"OUPS... l'argument de ""Galilée"" pour étayer se...",1.0,1,NaN,2019-01-23 00:36:26
3,Julien Ailloud,@JulienAilloud,1.548200e+09,NaN,Rien d'étonnant : blablater dans le vent pour ...,NaN,NaN,NaN,2019-01-23 00:27:04
4,Marien Lovichi,@MarienLovichi,1.548199e+09,NaN,"Vous habitez à #Dijon ? Vendredi, le #MoDem a...",1.0,7,4,2019-01-23 00:20:51


In [37]:
words_time_tweeter = dict()
for i, row in tweeter.iterrows():
    time_tweet = row["UNIXTIME"]
    if not isinstance(time_tweet, datetime.datetime):
        continue
    if not isinstance(row["message"], str):
        continue
    # for word in word_tokenize(row["message"], language='french'):
    for word in nlp(row["message"]):
        # word = word.lower()
        word = word.lemma_
        if word in words_time_tweeter:
            words_time_tweeter[word].append(time_tweet)
        else:
            words_time_tweeter[word] = [time_tweet]

In [18]:
[(x[0], len(x[1])) for x in sorted(words_time_tweeter.items(), key=lambda x: -len(x[1]))[:500]]

[('#', 48401),
 ('de', 15744),
 (',', 14102),
 (':', 11607),
 ('@', 11519),
 ('.', 11212),
 ('le', 11189),
 ('la', 9997),
 ('granddebat', 9587),
 ('les', 8916),
 ('à', 7871),
 ('et', 7767),
 ('des', 6284),
 ('’', 5944),
 ('granddébat', 5620),
 ('pour', 5047),
 ('!', 5007),
 ('…', 5002),
 ('du', 5000),
 ('en', 4637),
 ('?', 4125),
 ('https', 4009),
 ('un', 3979),
 ('sur', 3627),
 ('pas', 3566),
 ('que', 3531),
 ('est', 3530),
 ('dans', 3131),
 ('macron', 3120),
 ('une', 3005),
 ('au', 2995),
 ('giletsjaunes', 2931),
 ('ce', 2870),
 ('qui', 2661),
 ("''", 2654),
 ('l', 2553),
 ('a', 2497),
 ('il', 2457),
 ('...', 2456),
 ('``', 2363),
 ('avec', 2332),
 ('ne', 2118),
 ('on', 2020),
 ('vous', 2001),
 ('débat', 1953),
 ('d', 1750),
 ('par', 1711),
 ("c'est", 1647),
 ('je', 1596),
 ('plus', 1453),
 (')', 1336),
 ('aux', 1314),
 ('granddebatnational', 1304),
 ('«', 1302),
 ('»', 1280),
 ('se', 1279),
 ('nous', 1263),
 ('mais', 1259),
 ('sont', 1225),
 ('grand', 1177),
 ('(', 1174),
 ('http', 

In [19]:
iplot([go.Histogram(x=words_time_tweeter["ric"])])

In [20]:
iplot([go.Histogram(x=words_time_tweeter["européennes"]), go.Histogram(x=words_time_gdn["européennes"])])

In [21]:
word = "macron"
iplot([go.Histogram(x=words_time_tweeter[word], name="Tweeter"),
       go.Histogram(x=words_time_gdn[word], name="GDN")])

In [22]:
words_time_tweeter_day = dict()
for word in words_time_tweeter:
    words_time_tweeter_day[word] = [x.date()
                                    for x in words_time_tweeter[word] if not pd.isna(x)]

In [23]:
words_time_gdn_day = dict()
for word in words_time_gdn:
    words_time_gdn_day[word] = [x.date()
                                    for x in words_time_gdn[word] if not pd.isna(x)]

In [24]:
word = "mosquée"
iplot([go.Histogram(x=words_time_tweeter_day[word], name="Tweeter"),
       go.Histogram(x=words_time_gdn_day[word], name="GDN")])

In [38]:
# Weighted by retweet
words_time_tweeter2 = dict()
for i, row in tweeter.iterrows():
    time_tweet = row["UNIXTIME"]
    if not isinstance(time_tweet, datetime.datetime):
        continue
    if not isinstance(row["message"], str):
        continue
    if pd.isna(row["UNIXTIME"]):
        continue
    time_tweet = time_tweet.date()
    # for word in word_tokenize(row["message"], language='french'):
    for word in nlp(row["message"]):
        # word = word.lower()
        word = word.lemma_
        if pd.isna(row["n_retweet"]) or not row["n_retweet"].isnumeric():
            if word in words_time_tweeter2:
                words_time_tweeter2[word].append(time_tweet)
            else:
                words_time_tweeter2[word] = [time_tweet]
        else:
            if word in words_time_tweeter2:
                words_time_tweeter2[word] += [time_tweet] * (int(row["n_retweet"]) + 1)
            else:
                words_time_tweeter2[word] = [time_tweet] * (int(row["n_retweet"]) + 1)

In [52]:
word = nlp("benalla")[0].lemma_
iplot([go.Histogram(x=words_time_tweeter2[word], name="Tweeter"),
       go.Histogram(x=words_time_gdn_day[word], name="GDN")])